# Multimodal

## Setup

### Imports

In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import random
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, pipeline
from torch.utils.data import Dataset, DataLoader
from src.utils import *
from src.models import *
from src.process_reports import *
from src.train import train_mm, kfold_cv

%load_ext autoreload
%autoreload 2

### Set seed & device

In [3]:
set_seed(42)      
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# set torch matmul precision
torch.set_float32_matmul_precision('medium')

## Process reports

### Summarize reports

In [7]:
# remove invalid reports
reports_dir = 'data/tcga_crc/reports/'
for report in os.listdir(reports_dir):
    with open(reports_dir + report, 'r') as f:
        report_text = f.read()
        if len(report_text) < 20:
            print(f'invalid report: {report}\ncontent: {report_text}\ndeleting...\n')
            os.remove(reports_dir + report)

In [ ]:
%%script false --no-raise-error
# summarize report using gpt-3
summarize_reports(reports_dir='data/tcga_crc/reports', reports_sum_dir='data/tcga_crc/reports_sum')

### Extract text feats from reports

In [3]:
# %%script false --no-raise-error
# extract text feats from (summarized) reports
extract_text_feats(reports_dir='data/tcga_crc/reports_sum', report_feats_dir='data/tcga_crc/report_feats')

100%|██████████| 607/607 [04:33<00:00,  2.22it/s]


### Annotate subtype & grade from path reports

In [ ]:
%%script false --no-raise-error
lm_name = 'gpt-3.5-turbo'

# sample_report_path = 'data/reports_distilled/TCGA-WT-AB41.txt'
reports_dir = 'data/reports_distilled'
# create prompt
# prompt = create_zs_prompt(sample_report_path)

# args for generation
gen_args = {'max_tokens': 200}

# out = gen_subtype_grade_zs(lm_name, prompt, api='openai', args=gen_args)
df_res = classify_reports_zs(lm_name, reports_dir, api='openai', args=gen_args)
df_res.head(10)

## Task: Predict target from WSIs & reports

### Load data

In [29]:
# create dataloaders
wsi_feats_dir = 'data/tcga_crc/wsi_feats'
report_feats_dir = 'data/tcga_crc/report_feats'
target = 'msi'
data_file = 'data/tcga_crc/tcga_crc_gdc_pca.csv'
bsz = 32 # batch size for dataloaders
resample = True # resample data to balance classes
train_loader, val_loader, test_loader = create_dataloaders(target, data_file, wsi_feats_dir, report_feats_dir, use_rand_splits=True, bsz=bsz, resample=resample)

size of train set: 405, val set: 51, test set: 51
# samples for each class in train set: (array([0, 1, 2]), array([343,  13,  49]))
# samples for each class in val set: (array([0, 1, 2]), array([41,  4,  6]))
# samples for each class in test set: (array([0, 1, 2]), array([42,  3,  6]))


### Train & eval

In [36]:
# %%script false --no-raise-error
# init model
mode = 'mm'  # input modalities: 'text', 'img', or 'mm'
split = 'rand'   # dataset split: 'def' or 'rand'
num_classes = 3 # # classes for classification
class_weights = [1, 1, 1] # class weights for loss function
metrics = ['acc', 'bal_acc', 'f1', 'auroc', 'acc_per_class', 'f1_per_class', 'auroc_per_class'] # metrics to track

model = Attention1DClassifier(target=target, mode=mode, num_classes=num_classes, class_weights=class_weights, metrics=metrics) # model architecture: 'Attention1DRegressor' or 'Attention1DClassifier'

# set training args
args = {'num_epochs': 100, 'ckpt_name': f'ckpt_best_{mode}_{split}_split', 'resume_ckpt': None, 'tblog_name': f'best_{mode}_{split}_split'}

# train model
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
model, trainer = train_mm(model, train_loader, val_loader, args)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type               | Params
-------------------------------------------------------
0 | attention       | Sequential         | 262 K 
1 | classifier      | Sequential         | 9.2 K 
2 | loss            | CrossEntropyLoss   | 0     
3 | acc             | MulticlassAccuracy | 0     
4 | bal_acc         | MulticlassAccuracy | 0     
5 | f1              | MulticlassF1Score  | 0     
6 | auroc           | MulticlassAUROC    | 0     
7 | acc_per_class   | MulticlassAccuracy | 0     
8 | f1_per_class    | MulticlassF1Score  | 0     
9 | auroc_per_class | MulticlassAUROC    | 0     
-------------------------------------------------------
271 K     Trainable params
0         Non-trainable params
271 K     Total params
1.086     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 1.120


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 5 records. Best score: 0.722. Signaling Trainer to stop.


training on device: cpu


In [37]:
# evaluate the trained model on the test set
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  test_acc_class_0_epoch   │           0.625           │
│  test_acc_class_1_epoch   │    0.1666666716337204     │
│  test_acc_class_2_epoch   │    0.4000000059604645     │
│      test_acc_epoch       │    0.6078431606292725     │
│ test_auroc_class_0_epoch  │    0.4635416865348816     │
│ test_auroc_class_1_epoch  │    0.3620689809322357     │
│ test_auroc_class_2_epoch  │     0.800000011920929     │
│     test_auroc_epoch      │    0.5761451125144958     │
│    test_bal_acc_epoch     │    0.4418300688266754     │
│   test_f1_class_0_epoch   │     0.728560209274292     │
│   test_f1_class_1_epoch   │           0.125           │
│   test_f1_class_2_epoch   │    0.2666666805744171     │
│       test_f1_epoch       │    0.4028104841709137     │
│      test_loss_epoch      │    0.8762999773025513     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc_epoch': 0.6078431606292725,
  'test_f1_epoch': 0.4028104841709137,
  'test_bal_acc_epoch': 0.4418300688266754,
  'test_auroc_epoch': 0.5761451125144958,
  'test_acc_class_0_epoch': 0.625,
  'test_acc_class_1_epoch': 0.1666666716337204,
  'test_acc_class_2_epoch': 0.4000000059604645,
  'test_f1_class_0_epoch': 0.728560209274292,
  'test_f1_class_1_epoch': 0.125,
  'test_f1_class_2_epoch': 0.2666666805744171,
  'test_auroc_class_0_epoch': 0.4635416865348816,
  'test_auroc_class_1_epoch': 0.3620689809322357,
  'test_auroc_class_2_epoch': 0.800000011920929,
  'test_loss_epoch': 0.8762999773025513}]

### K-fold CV

In [5]:
# %%script false --no-raise-error
# run k-fold CV
# init model
data_file = 'data/data_tcga_brca_sg_pca.csv'
target = 'msi'
mode = 'mm'
dataset = MMDataset(target, data_file)
bsz = 64
model_class = Attention1DRegressor
# model args
model_args = {'mode': mode, 'target': target}
# train args
train_args = {'bsz': bsz, 'k': 5, 'num_epochs': 100, 'patience': 5, 'save_top_k': 0, 'tblog_name': f'best_{mode}_kfold', 'enable_progress_bar': False}

res_kfold_cv = kfold_cv(model_class, dataset, model_args, train_args)
metrics = ['test_corr_epoch', 'test_r2_epoch']
avg_res = {k: np.mean([res[k] for res in res_kfold_cv]).round(3) for k in metrics}
print(f"avg res over {train_args['k']} folds: {avg_res}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


training fold 1/5


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type            | Params
----------------------------------------------
0 | attention | Sequential      | 262 K 
1 | regressor | Sequential      | 3.1 K 
2 | loss      | MSELoss         | 0     
3 | corr      | PearsonCorrCoef | 0     
4 | r2        | R2Score         | 0     
----------------------------------------------
265 K     Trainable params
0         Non-trainable params
265 K     Total params
1.062     Total estimated model params size (MB)
Metric val_loss improved. New best score: 0.003
Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New b

training on device: cpu


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_corr_epoch      │    0.27596205472946167    │
│       test_r2_epoch       │   -0.18922992050647736    │
└───────────────────────────┴───────────────────────────┘

...

training fold 5/5


Metric val_loss improved. New best score: 0.009
Metric val_loss improved by 0.005 >= min_delta = 0.0. New best score: 0.004
Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.003
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.002
Metric val_loss improved by 0.000 >= min

training on device: cpu


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_corr_epoch      │    0.18666738271713257    │
│       test_r2_epoch       │   -0.03658778592944145    │
└───────────────────────────┴───────────────────────────┘

avg res over 5 folds: {'test_corr_epoch': 0.219, 'test_r2_epoch': -0.119}
